In [ ]:
!pip install -q datasets
!pip install -q sentencepiece
!pip install -q transformers
!pip install -q sacremoses



---



In [ ]:
import os 
from itertools import islice

import torch

from datasets import load_dataset, Dataset
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

In [ ]:
import torch

torch.__version__

'1.13.1+cu116'



---



In [ ]:
model_name="Helsinki-NLP/opus-mt-tc-big-en-pt"

In [ ]:
pipe = pipeline("translation", model=model_name, device=0)



---



In [ ]:
ds = load_dataset("json", data_files="alpaca_data.json")



---



In [ ]:
items_to_translate = []

In [ ]:
for row in tqdm(ds["train"], total=len(ds["train"])):

    instruction = f">>por<< {row['instruction']}" if row["instruction"] else ""
    input = f">>por<< {row['input']}" if row["input"] else ""
    output = f">>por<< {row['output']}" if row["output"] else ""

    items_to_translate.append({"text": instruction})
    items_to_translate.append({"text": input})
    items_to_translate.append({"text": output})

In [ ]:
ds_to_translate = Dataset.from_list(items_to_translate)

In [ ]:
ds_to_translate



---



In [ ]:
translated_ds = []

In [ ]:
for out in tqdm(
    pipe(
            KeyDataset(ds_to_translate, "text"), 
            batch_size=32,
            truncation=True,
            max_length=512
        ),
    total = len(ds_to_translate)
    ):
    translated_ds.extend(out)

In [ ]:
len(translated_ds)

In [ ]:
translated_ds = Dataset.from_list(translated_ds)



---



In [ ]:
def batched(iterable, n):
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while (batch := tuple(islice(it, n))):
        yield batch

In [ ]:
ds_data = []

In [ ]:
for batch in batched(ds, 3):
    ds_data.append(
        {
            "instruction": batch[0]["translation_text"],
            "input": batch[1]["translation_text"],
            "output": batch[2]["translation_text"],
        }
    )

In [ ]:
ds = Dataset.from_list(ds_data)

In [ ]:
ds.push_to_hub("user/alpaca-data-pt-br")